### Setup - Download

In [ ]:
!rm -f watson_streaming_pipelines.py*
!wget https://raw.githubusercontent.com/raanonr/DSX/master/Notebooks/watson_streaming_pipelines.py
# You may need this:
#!pip install kafka

### Setup - Credentials and MH topic

In [ ]:
# @hidden_cell

# The Object Storage credentials should NOT come from the 'Service credentials' page.
# Get them from the option in DSX Notebook to 'insert to code' from the Data -> Connections.
# Then ADD an entry for the service's PUBLIC endpoint.

cos_credentials = {
  'iam_url':'<REPLACE>',
  'api_key':'<REPLACE>',
  'resource_instance_id':'<REPLACE>',
  'url':'<REPLACE>',
      'endpoint':'<REPLACE>'
}

# The Message Hub credentials should come from the 'Service credentials' page.

mh_credentials = {
  "kafka_brokers_sasl": [
      "<REPLACE>"
  ],
  "user": "<REPLACE>",
  "password": "<REPLACE>"
}

# Enter your Message Hub topic

mh_topic = '<REPLACE>'

### Load the yelp data

In [ ]:
import watson_streaming_pipelines as stp

bucket_name = 'pyml'
object_name = 'yelp_academic_dataset_review_first200k.json'

data = stp.get_from_cloud_object_storage( 
            api_key = cos_credentials['api_key'],
            full_object_path = bucket_name + '/' + object_name, 
            auth_endpoint = cos_credentials['iam_url'],
            service_endpoint = cos_credentials['endpoint']
) 

# Verify
print(len(data))

### Convert and decode the binary data and load the yelp List with the JSON objects

In [ ]:
from io import StringIO
import json

yelp = []
for line in StringIO(data.decode("utf-8")):
    yelp.append( json.loads(line))

# Verify
print(yelp[0]['text'])

### Create the Message Hub producer

In [ ]:
producer = stp.create_messagehub_producer( username = mh_credentials['user'], password = mh_credentials['password'], kafka_brokers_sasl = mh_credentials['kafka_brokers_sasl'])

### Send selected yelp fields to the MH topic

In [ ]:
import time
print('Yelp entries:', format(len(yelp), ',d'))

n=0
for msg in yelp: #[:1000]:
    #producer.send( mh_topic, { 'stars': msg['stars'], 'text': msg['text']})
    n += 1
    if (n % 1000) == 0:
        time.sleep(1)
        print(n, "", end="")